
#Week 2 - Manipulating Structured Data

(Be sure to Copy to Drive first)

This will build on content introduced in week 1. In week 1 we learned that information can be presented on the web in HTML (ie, with formatting and graphics) or as data that can be read programmatically via Python through an _API_ (Application Program Interface).

This week we will build on what we learned by creating a dataset from a few different APIs.


# CrossRef

CrossRef is one of the organizations that product DOIs. These are the unique serial numbers that get added to published works. I think the biggest advantage to DOIs is that they allow us to get to full text of items even when URLs change for publisher websites. The other great thing is that CrossRef provides a bunch of other metadata alongside the URL to the full-text.

We'll grab information from the CrossRef API using a Python library called [crossrefapi](https://github.com/fabiobatalha/crossrefapi)


## CrossRef via the Web

Head over to the [web search](https://search.crossref.org/) for CrossRef and experiment with some searches. You'll see that there is a lot of different fields in the data.


# CrossRef via an API

We can interact with CrossRef via an API as well. This will just return data in a way that can be treated like a JSON variable.


# One item two way

To really highlight the difference between the web view and the API check out these two links for a piece I wrote about data analysis:

- [https://api.crossref.org/works/10.5334/johd.109](https://api.crossref.org/works/10.5334/johd.109)


- [https://doi.org/10.5334/johd.109](https://doi.org/10.5334/johd.109)

Click through on both to see what happens

# CrossRef via API and Python

In order to make working with the API more manageable we'll use a Python library called [crossrefapi](https://github.com/fabiobatalha/crossrefapi) that will allow us to use Python variables instead of manipulating JSON directly. Since this library is not part of the usual Python deployment we will need to install it specifically. Details about installing libraries are in the week 3 warmup activity in case you are curious

When you first run this code cell it might ask you to 'Restart' by clicking a button. Please do that, and re-run the cell. This types of things sometimes happen when you start including new libraries in the Colab environment.

In [ ]:
#install the library
!pip install crossrefapi


#Load in our other usual libraries
import pandas as pd
from google.colab import drive
import numpy as np

print("Ready to proceed!")

With the library installed we will now query the API to look at the same article we saw above.

In [ ]:
from crossref.restful import Works
works = Works()
item = works.doi("10.5334/johd.109")

Run the next cell to see how the information looks a lot like JSON.

In [ ]:
item


# Parts of the data

By doing a key lookup, we can find the associated value

In [ ]:
item['title']

**Q1** Print the ISSN that the article is from

In [ ]:
#Q1
item[]

**Q2** Print the number of references this article has

In [ ]:
#Q2
item[]

**Q3** Print the volume that this article is from

In [ ]:
#Q3
item[]

# A more complex part of the data

Sometime we can use a key to get a value that is actually a more complex thing, like a list.

Check out this example of the author information.

In [ ]:
item['author']


# Two levels deep

For example we can find the `family` name of the author with something like the following

In [ ]:
#We are interested in the first item in this list
#which is why we index at 0
#if this piece had more than 1 author the list above
#would be bigger
author_info = item['author'][0]
author_info['family']


**Q4** Print the ORCID for this author

In [ ]:
author_info = item['author'][0]
author_info['']

**Q5** Print the list of references this article has. You just need to modify line 1.

In [ ]:
references = item['']

for reference in references:
  #Even more JSON objects!
  print(reference)


# Two steps

We are now going to try a search in the CrossRef API, find some journal articles and identify the ORCIDs of the authors. We are then going to beef up this information with the ORCID API.


# Searching the API

We'll search the API based on a search query that we specify in our next cell.

In [ ]:
search_string = "ACRL Competencies and student success"

In [ ]:
search_results = works.query(bibliographic=search_string)

#We can see what actual URL of that we'll use by adding .url to our variable name
print(search_results.url)

# Filtering & Select the results

Now with our query variable put together we'll actually search the API and _filter_ our results to:

- Only journal articles, line 7
- One of the authors has an ORCID, line 8 (more on ORCIDs in a bit)

and then we will _select_ only the fields we are interested in. Those are specified in the list on line 2.

In [ ]:

select_fields = ['DOI','score','references-count','title','author']
number_of_results = 20

results_for_df = []

for result in search_results.filter(type="journal-article", \
                                    has_orcid="1", \
                                    ).select(select_fields).sample(number_of_results):

                                    print("Retrieved: \n",result)

                                    doi = result['DOI']
                                    score = result['score']
                                    references_count = result['references-count']
                                    title = result['title'][0]

                                    #We will 'try' to get the ORCID of the first listed author
                                    #if that throws an error (except) we'll just leave it blank
                                    #more details on this in a couple of cells
                                    try:
                                      orcid = result['author'][0]['ORCID']
                                    except:
                                      orcid = ""

                                    results_for_df.append([doi,score,references_count,title,orcid])




# Data to Dataframe

The next cell will turn our retrieved data in a dataframe and display it to the screen.

In [ ]:
doi_data = pd.DataFrame(results_for_df,columns=select_fields)
doi_data


#Missing ORCIDs?

Out code is very simple. We are only grabbing the ORCID of the first author that is listed in the item. That author might not have an ORCID. Our search only guaranteed at least one of the authors had and ORCID.

To make life easier we are going to drop any lines in our dataframe that are missing orcids with the following cell

In [ ]:
doi_data = doi_data[doi_data['author']!=""]
doi_data

# ORCID Data

You might of heard of ORCID before. It is basically a service that allows a researcher to put a unique serial number on works they publish.

The good news is that ORCID has an API as well. But with this one you need to register ahead of time and get an API key and secret. These basically tell the API who are are when you are using it.

This will require a new library to be installed.

In [ ]:
!pip install orcid

import orcid

print("Ready to proceed!")


# API again, but with more information

We need to register with the ORCID API before we use it. We need to get an API ID and Secret ([details here](https://info.orcid.org/what-is-orcid/services/public-api/)).

When you generate them simply fill in these details in line 2 and 3. If you are having trouble with this please get in touch with me and I can help you with that process.

In [ ]:
# Our Info from the ORCID site
Client_ID = ""
Client_Secret = ""

#with that set we do our Python API work...
orcid_api = api = orcid.PublicAPI(Client_ID, Client_Secret, sandbox=False)
search_token = api.get_search_token_from_orcid()


# Grabbing Info

We can now grab information about an ORCID. Run the next cell to see the [works](https://orcid.org/0000-0001-9229-8569#works) I've produced.

In [ ]:
orcid_results = orcid_api.read_record_public("0000-0001-9229-8569",'works', search_token)
orcid_results

#JSON overload!

That's a lot of JSON to look at. What we are going to focus on is just grabbing the number of _Works_ each author has published and use that value in our analysis.

In [ ]:
#The number of works I've published
len(orcid_results['group'])


# Filling our dataset

Now we are going to put all of this together!

We are going to use the ORCID API to find out how many things each first author has published and then we are going to add this to our dataframe!

Why? You'll see soon.

In [ ]:
authors = doi_data['author'].to_list()
author_works_count = []

for a in authors:
  author_orcid = a.split('/')[-1]
  print("Working on",a)
  orcid_results = orcid_api.read_record_public(author_orcid,'works', search_token)
  count = len(orcid_results['group'])
  print("Found ", count, "works")
  author_works_count.append(count)


# Augmenting our Dataframe

Now that we have this information, we'll add it as another column in our dataframe. This cell might throw a _warning_ but that is safe to ignore.

In [ ]:
#Now we add this list as a column to our Dataframe
doi_data['author_work_count'] = author_works_count
doi_data


# The Grand Finale!

We are going to now make a correlation matrix to see what relationship there is between score, number of references, and the number of works the author has completed!

In [ ]:
#Lot going on here... Just run this cell and look at the final product to answer the next question
corr = doi_data.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 )


# Congrats on doing Data Science

I'll let you decide what to think of the results from the correlation matrix.

Remember
- -1 is strong negative correlation, eg. One goes up the other always goes down.
- 0 means no correspondence between the two variables
- 1 a strong positive correlation, eg. One goes up the other always goes up too.



# Saving our new dataset!

We've spent some time creating a nice dataset from a few different sources so let's take the time to save it as a CSV file. Like we did in the warm-up we'll connect to Google Drive and save our file in our usual folder.



In [ ]:
#This will automatically download the file
#please check your downloads directory for it.
from google.colab import drive
drive.mount('/content/gdrive')
doi_data.to_csv('adv_week2_doi_dataset.csv',index=False)

#This next line copies (cp) the csv file from our working folder
#to the special folder we've created for our work
!cp adv_week2_doi_dataset.csv /content/gdrive/MyDrive/LibraryJuicePython

print("File copied to Drive!")


# Questions for this Workalong

You'll notice there is a lack of questions in this workalong. I'm going to ask you now to head up to the section called **Searching the API** and change the search query to something you are interested in and re-run the subsequent cells to create a dataset on a different topic. Feel free to also adjust `number_of_results` to a number larger than 20 to make a bigger dataset (but don't go too big with it, certainly keep in under 50)

Be sure to save the CSV file of the final dataset in Google Drive. We'll be using it again later.